# Linear regression PyTorch

In [48]:
import sys
from pathlib import Path
sys.path.append(str(Path("..")))

# imports
from common.utils import get_numeric_data, split

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import torch

In [49]:
data = get_numeric_data()
data

,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,122.0,127.3,20,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74
1,160.0,142.5,19,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79
2,122.0,124.8,19,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74
3,122.0,119.6,20,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12
4,100.0,141.5,45,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,125.0,122.2,19,0,6,7,5,13.600000,0,0,6,8,5,12.666667,0,15.5,2.8,-4.06
4420,120.0,119.0,18,0,6,6,6,12.000000,0,0,6,6,2,11.000000,0,11.1,0.6,2.02
4421,154.0,149.5,30,0,7,8,7,14.912500,0,0,8,9,1,13.500000,0,13.9,-0.3,0.79
4422,180.0,153.8,20,0,5,5,5,13.800000,0,0,5,6,5,12.000000,0,9.4,-0.8,-3.12


In [50]:
X = data.drop("Admission grade", axis=1)
X = StandardScaler().fit_transform(X)       # normalize
X = torch.tensor(X, dtype=torch.float32)
y = data["Admission grade"]
y = torch.tensor(y, dtype=torch.float32)

X_train, X_val, X_test, y_train, y_val, y_test = split(X, y)

In [51]:
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize):
        super().__init__()
        self.linear = torch.nn.Linear(inputSize, 1)     # linear layer       y = w1*x1 + w2*x2 + ... + b

    def forward(self, x):
        return self.linear(x)

In [ ]:
def train_and_eval(device, X_train, y_train, X_val, y_val, X_test, y_test):  
    
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)
      
    model = LinearRegression(X_train.shape[1]).to(device)
    criterion = torch.nn.MSELoss()                          # MSE
    optimizer = torch.optim.SGD(model.parameters())         # gradient descent implementation

    for _ in range(3000):
        y_pred = model(X_train)                # X * W^T + b
        loss = criterion(y_pred, y_train)      # 1/n ∑ (ytrue - ypred)^2       (MSE)
        
        optimizer.zero_grad()                   # clear gradient from previous steps
        loss.backward()                         # back propagration - calculates gradients of the loss 
        
        optimizer.step()                        # w = w - learning_rate * (∂loss / ∂w)
        
    model.eval()                    # switch to eval mode
    
    with torch.no_grad():           # no gradient in testing phasse
        y_pred_train = model(X_train).cpu().numpy()
        y_pred_val = model(X_val).cpu().numpy()
        y_pred_test = model(X_test).cpu().numpy()

        
    mse_train = mean_squared_error(y_train.cpu().numpy(), y_pred_train)
    mse_val = mean_squared_error(y_val.cpu().numpy(), y_pred_val)
    mse_test = mean_squared_error(y_test.cpu().numpy(), y_pred_test)
    
    print(f"MSE train: {mse_train}")
    print(f"MSE validation: {mse_val}")
    print(f"MSE test: {mse_test}")
        

CPU

In [53]:
train_and_eval(torch.device("cpu"), X_train, y_train, X_val, y_val, X_test, y_test)

c:\ProgramData\anaconda3\envs\msid2_pytorch\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([3096])) that is different to the input size (torch.Size([3096, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


MSE train: 204.5994873046875
MSE validation: 223.5696563720703
MSE test: 220.15664672851562


GPU

In [54]:
if torch.cuda.is_available():
    train_and_eval(torch.device("cuda"), X_train, y_train, X_val, y_val, X_test, y_test)
else:
    print("Not available")

c:\ProgramData\anaconda3\envs\msid2_pytorch\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([3096])) that is different to the input size (torch.Size([3096, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.